In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F

from einops import rearrange
#from torchsummary import summary

import sys
sys.path.insert(0, '../helpers')
# sys.path.insert(0,'/home/labs/rudich/sagima/helpers')
from plotting import *
from validate import *
from models import *
from data_handlers import *
from training import *
from metrics import *
from explore import *
from torchsummary import *
from visual_helpers_2 import *

#from visual_helpers import *

import itertools



#taken from the model building section - 
import pandas as pd
pd.set_option('max_colwidth', 800)


import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import xgboost as xg
from statsmodels.tools.eval_measures import rmse
import scipy 
from statsmodels.tsa.stattools import grangercausalitytests
import torch.optim as optim
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.fft import fft, fftfreq
import folium

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
import cartopy.feature


import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio


import ast
from scipy.spatial.distance import cdist

from tqdm import tqdm


/home/labs/rudich/sagima/.conda/envs/sagi_base/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/labs/rudich/sagima/.conda/envs/sagi_base/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def convert_string_to_tuple_list(string_list):
    tuple_list = []
    for string in string_list:
        # Remove leading and trailing whitespace and parentheses
        string = string.strip('()')
        # Convert string to tuple using ast.literal_eval
        tuple_value = ast.literal_eval(string)
        tuple_list.append(tuple_value)
    return tuple_list

with open("saved_turkey_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_turkey_lat_lon_cords = pickle.load(fp)

with open("saved_turkey_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_turkey_lat_lon_idx = pickle.load(fp)
    
with open("saved_greece_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_greece_lat_lon_cords= pickle.load(fp)

with open("saved_greece_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_greece_lat_lon_idx = pickle.load(fp)
    
saved_turkey_lat_lon_cords = convert_string_to_tuple_list(saved_turkey_lat_lon_cords)
saved_turkey_lat_lon_cords.append((35.0,27.0))
saved_turkey_lat_lon_cords.append((35.0, 45.5))

with open("saved_turkey_and_greece_lat_lon_cords", "rb") as fp:   # Unpickling
    saved_turkey_and_greece_lat_lon_cords= pickle.load(fp)

with open("saved_turkey_and_greece_lat_lon_idx", "rb") as fp:   # Unpickling
    saved_turkey_and_greece_lat_lon_idx = pickle.load(fp)


In [3]:

colab_dir = "/home/labs/rudich/Collaboration/dust_prediction/data"
meteo_dir = colab_dir+"/interpolated_mean_normalized/"
meteo_de_dir = colab_dir+"/meteorology_renormalization_tensors"
dust_dir = colab_dir+"/dust_61368_108_2_339/"

pm10_all = torch.load(meteo_de_dir+"/meteorology_tensors_1_81_189_interpolated_mean_normalized_pm10_tensor_denormalized.pkl")
labels_all = torch.load(dust_dir+"dust_61368_108_2_339_full_tensor.pkl")

timestamps_labels = torch.load(dust_dir+"dust_61368_108_2_339_full_timestamps.pkl")
timestamps_mpm = pd.to_datetime(torch.load(colab_dir+"/meteorology_tensors_1_81_189_general_timestamps.pkl"))
timestamps_all = pd.to_datetime(torch.load(colab_dir+"/meteorology_tensors_1_81_189_general_timestamps.pkl"))




In [4]:
colab_dir = "/home/labs/rudich/Collaboration/dust_prediction/data"
meteo_dir = colab_dir+"/interpolated_mean_normalized/"
meteo_de_dir = colab_dir+"/meteorology_renormalization_tensors"
dust_dir = colab_dir+"/dust_61368_108_2_339/"

meteo_meta = torch.load(meteo_dir+"/meteorology_tensor_62_81_189_metadata.pkl")
dust_meta = torch.load(dust_dir+"/dust_61368_108_2_339_metadata.pkl")

lons_dict = meteo_meta['dims']['lons']
lats_dict = meteo_meta['dims']['lats']



In [5]:
df_pm0 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_0.csv",index_col=0).dropna(axis=1,how='all')
df_pm1 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_1.csv",index_col=0).dropna(axis=1,how='all')
df_pm2 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_2.csv",index_col=0).dropna(axis=1,how='all')
df_pm3 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_3.csv",index_col=0).dropna(axis=1,how='all')
df_pm4 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_4.csv",index_col=0).dropna(axis=1,how='all')
df_pm5 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_5.csv",index_col=0).dropna(axis=1,how='all')
df_pm6 = pd.read_csv("/home/labs/rudich/sagima/TOAR/pm_6.csv",index_col=0).dropna(axis=1,how='all')

df_pm =(
        pd.merge(df_pm0, df_pm1, how='outer',left_index=True, right_index=True)
        .pipe(lambda d:pd.merge(d, df_pm2, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm3, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm4, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm5, how='outer',left_index=True, right_index=True))
        .pipe(lambda d:pd.merge(d, df_pm6, how='outer',left_index=True, right_index=True))


)
df_cord = pd.read_csv("/home/labs/rudich/sagima/TOAR/coordinates_pm.csv")
df_cord = df_cord.rename(columns={df_cord.columns[0]:'station_id'})


first_year = int(df_pm.index[0][:4])
last_year = int(df_pm.index[-1][:4])

timestamps = df_pm.index.values.astype('datetime64[s]').astype(np.int64)
stn_ids = df_pm.columns.values.astype(np.int64)


pm10_values = df_pm.to_numpy()

pm10_tensor = pm10_values.reshape(len(timestamps), len(stn_ids), 1)

# Convert the tensor to a PyTorch tensor
labels_all = torch.from_numpy(pm10_tensor)

timestamp_labels = pd.to_datetime(df_pm.index)

zero_var = (pm10_all.var((2,3))==0).reshape(-1).numpy()
pm10_valid_timestamps = timestamps_all[~zero_var]
pm10_valid_all = pm10_all[~zero_var]

l = [pm10_valid_timestamps, timestamp_labels]
shared_timestamps = l[0].intersection(l[1])

shared_timestamps_idxs = []
for timestamps_list in tqdm(l):
    shared_idxs_per_list = []
    for shared_t in shared_timestamps:
        try:
            shared_idxs_per_list.append(timestamps_list.get_loc(shared_t))
            
        except:
            print(f"Error! Something wierd happend with {shared_t} and {timestamps_list}. Aborting")
        if shared_t == shared_timestamps[-1]:
            print(max(shared_idxs_per_list))
    shared_timestamps_idxs.append(shared_idxs_per_list)

inputs_all = pm10_valid_all
labels_all = labels_all



years = [ i for i in range(first_year, last_year)]
months = [i for i in range(1,13)]

dt = pd.Series(shared_timestamps).dt
years = np.isin(dt.year, years)
months = np.isin(dt.month, months)
include = (years) * (months)

pm10_shared_timestamps = shared_timestamps[include]
inputs = inputs_all[shared_timestamps_idxs[0]][include]
labels = labels_all[shared_timestamps_idxs[1]][include]



/tmp/ipykernel_30159/2475775844.py:26: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamps = df_pm.index.values.astype('datetime64[s]').astype(np.int64)
 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  2.60it/s]

52513


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  2.15it/s]

61367


In [6]:
channels46 = ["Q250","Q500","Q700","Q850","Q900",
            "T250","T500","T700","T850","T900",
            "U250","U500","U700","U850","U900",
            "V250","V500","V700","V850","V900",
            "Z250","Z500","Z700","Z850","Z900",
            "PV250","PV500","PV700","PV850","PV900",
            "OMEGA250","OMEGA500","OMEGA700","OMEGA850","OMEGA900",
            "SLP","u10","v10","duaod550",'aermr06_50','aermr06_40','aermr06_30', 'aermr06_20',
              "pm2p5","pm10","tcwv"]


In [7]:
all_stations_lonlat = (df_cord 
                       .rename(columns={'lng':'lon'})
                       .assign(stn_id=np.arange(0,df_cord.shape[0],1))
                        )


df_pm.index = pd.to_datetime(df_pm.index)
df_pm.columns =  df_pm.columns.astype(int)
set1 = set(pm10_valid_timestamps)
set2 = set(df_pm.index)

# Find the intersection of the sets
intersection = set1.intersection(set2)

# Convert the intersection set to a DatetimeIndex
common_timestamps = pd.DatetimeIndex(pd.to_datetime(list(intersection)))
common_timestamps = common_timestamps.sort_values()
# Find the index where the intersection starts
start_index = list(pm10_valid_timestamps).index(common_timestamps[0])

# Find the index where the intersection ends
end_index = list(pm10_valid_timestamps).index(common_timestamps[-1])


In [8]:
inv_lats = {v: k for k, v in lats_dict.items()}
inv_lons = {v: k for k, v in lons_dict.items()}

In [9]:
import pickle
with open('pixel_to_station_id.pkl','rb' ) as f:
       simple_dict =  pickle.load(f)

In [10]:
simple_dict

{'[34.5 33. ]': [28218],
 '[35. 33.]': [28220],
 '[35.  33.5]': [28219],
 '[35.5 24. ]': [46294],
 '[35.5 25. ]': [46239],
 '[36. 36.]': [6984, 8210, 56731, 56850],
 '[36.5 34. ]': [72510],
 '[36.5 36. ]': [56669],
 '[36.5 37. ]': [56646],
 '[37. 28.]': [5132],
 '[37.  28.5]': [6979, 8122, 56618],
 '[37.  30.5]': [56739, 56846],
 '[37. 33.]': [56679],
 '[37.  34.5]': [56738, 56797],
 '[37.  35.5]': [6968, 6971, 6983, 8178, 8211, 56603, 56604, 56653, 56718],
 '[37. 36.]': [56612],
 '[37.  36.5]': [56858],
 '[37.  37.5]': [56794],
 '[37. 39.]': [56785],
 '[37.5 28. ]': [4997, 56668],
 '[37.5 30.5]': [56702],
 '[37.5 37. ]': [56733, 56857],
 '[37.5 40.5]': [56656],
 '[37.5 42.5]': [56773],
 '[37.5 43.5]': [56666],
 '[38.  21.5]': [14311, 46261, 46264, 46271],
 '[38.  23.5]': [9937,
  12347,
  12405,
  12471,
  12501,
  46251,
  46253,
  46263,
  46269,
  46272,
  46273,
  46276,
  46277,
  46284,
  46285,
  46287,
  46290],
 '[38. 24.]': [9993,
  10162,
  12307,
  12353,
  12502,
  46247,

In [9]:
# import pickle
# with open('pixel_to_station_id.pkl','rb' ) as f:
#        simple_dict =  pickle.load(f)

# stations_tensor = np.zeros((pm10_valid_timestamps.shape[0], 1, len(lats_dict), len(lons_dict)))

# stations_tensor[:,0,:,:] = np.nan
# # Iterate over each key in the simple_dict
# for key in tqdm(simple_dict):
#     # Extract the latitude and longitude values from the key
#     values = key[1:-1].split()

#     # Convert the values to float
#     values = [float(value) for value in values]

#     lat = values[0]
#     lon = values[1]

#     # Get the corresponding indices from the inv_lats and inv_lons dictionaries
#     lat_index = inv_lats[lat]
#     lon_index = inv_lons[lon]

#     # Iterate over each timestamp in the common_timestamps list
#     for t, timestamp in enumerate(common_timestamps):
#         if t >= start_index:
#             # Get the corresponding row in the pm10_df DataFrame based on the timestamp
#             df_row = df_pm.loc[timestamp]

#             # Filter the columns in the DataFrame based on the values from the simple_dict
#             filtered_df = df_row[simple_dict[key]]

#             # Calculate the average value of the filtered columns
#             mean_value = filtered_df.mean()

#             # Update the tensor at the specified indices with the mean value
#             stations_tensor[t, 0, lat_index, lon_index] = mean_value


In [10]:
# torch.save(torch.from_numpy(stations_tensor), 'stations_tensor.pt')
stations_tensor = torch.load('stations_tensor.pt')

In [11]:

tensors_list = []
#labels_all = torch.load(dust_dir+"dust_61368_108_2_339_full_tensor.pkl")
for channel in tqdm(range(len(channels46))):
    data_dir = "/home/labs/rudich/Collaboration/dust_prediction/data/interpolated_mean_normalized/"
    dust_dir = "/home/labs/rudich/Collaboration/dust_prediction/data/dust_61368_108_2_339/"
    channel_all = torch.load(f"/home/labs/rudich/Collaboration/dust_prediction/data/meteorology_renormalization_tensors/meteorology_tensors_1_81_189_interpolated_mean_normalized_{channels46[channel]}_tensor_denormalized.pkl")
    tensors_list.append(channel_all)
stacked_tensor = torch.stack(tensors_list, dim=1)
stacked_tensor = stacked_tensor.squeeze(dim=2)
stacked_tensor = stacked_tensor[~zero_var]
print(stacked_tensor.shape)

100%|████████████████████████████████████████████████████████| 46/46 [02:09<00:00,  2.81s/it]


torch.Size([52514, 46, 81, 189])


In [12]:

# full = torch.cat((stacked_tensor,torch.from_numpy(stations_tensor)),dim=1)
full = torch.cat((stacked_tensor,stations_tensor),dim=1)
del stacked_tensor
del stations_tensor
del tensors_list
del channel_all

In [ ]:
a = torch.zeros(1, 81, 189)
a[0,10:75,50:117] = 1
print_parameter(a,0,figsize=(20,10))

In [13]:
# Define the slice indices
lat_start, lat_end = 10, 75
lon_start, lon_end = 50, 117

# Slice the tensor
sliced_tensor = full[:, :, lat_start:lat_end, lon_start:lon_end]

# Create new dictionaries for the sliced lat and lon indices
new_lats_dict = {i: lats_dict[i] for i in range(lat_start, lat_end)}
new_lons_dict = {i: lons_dict[i] for i in range(lon_start, lon_end)}

def replace_keys_with_sequential_integers(dictionary):
    new_dict = {}
    keys = list(dictionary.keys())
    for i in range(len(keys)):
        new_dict[i] = dictionary[keys[i]]
    return new_dict

new_lats_dict = replace_keys_with_sequential_integers(new_lats_dict)
new_lons_dict = replace_keys_with_sequential_integers(new_lons_dict)

del full


In [14]:
import pickle
with open('pixel_to_station_id.pkl','rb' ) as f:
       simple_dict =  pickle.load(f)

In [15]:

tensor = sliced_tensor
# Given information
tensor_shape = (52514, 47, 65, 67)

timestamps = common_timestamps # List of timestamps (length 52514)

# Reshape the tensor
reshaped_tensor = np.reshape(tensor, (tensor_shape[0], tensor_shape[1], -1))

# Initialize an empty list to store individual DataFrames
dfs = []

# Iterate over lat and lon combinations
for lat_index in tqdm(range(tensor_shape[2])):
    for lon_index in range(tensor_shape[3]):        
        # Create a DataFrame for the current lat-lon combination
        df = pd.DataFrame(reshaped_tensor[:, :, (lat_index * tensor_shape[3]) + lon_index])

        df['time'] = timestamps
        lat_loc = new_lats_dict[lat_index]
        lon_loc = new_lons_dict[lon_index]
        df['lat_lon_coord'] = str((float(lat_loc),float(lon_loc)))

        dfs.append(df)

# Concatenate all the individual DataFrames into one big DataFrame
df = pd.concat(dfs, ignore_index=True)

df


100%|████████████████████████████████████████████████████████| 65/65 [00:04<00:00, 14.72it/s]


,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,time,lat_lon_coord
0,0.000013,0.000474,0.001102,0.000835,0.001759,226.508026,263.001862,283.515350,291.670929,293.279907,...,2.311351e-08,2.772695e-11,6.839194e-12,3.462257e-11,5.610545e-08,7.052427e-08,4.557641,NaN,2003-01-01 00:00:00+00:00,"(15.0, -7.0)"
1,0.000012,0.000481,0.001061,0.000656,0.001770,226.306519,262.616394,283.216797,291.796021,292.988007,...,2.134657e-08,2.658108e-11,6.579950e-12,3.628391e-11,7.311485e-08,9.019969e-08,4.351818,NaN,2003-01-01 03:00:00+00:00,"(15.0, -7.0)"
2,0.000012,0.000364,0.001008,0.000889,0.001820,226.277267,262.529816,282.838684,291.154480,292.559235,...,1.957962e-08,2.543520e-11,6.320705e-12,3.794526e-11,9.012425e-08,1.098751e-07,4.145996,NaN,2003-01-01 06:00:00+00:00,"(15.0, -7.0)"
3,0.000010,0.000294,0.000910,0.000942,0.001694,226.691376,262.407745,282.387604,290.868073,292.393738,...,2.560189e-08,1.669112e-11,7.090138e-12,3.864344e-11,6.376614e-08,8.051371e-08,4.174264,NaN,2003-01-01 09:00:00+00:00,"(15.0, -7.0)"
4,0.000009,0.000176,0.000806,0.000866,0.001724,226.552704,263.003693,282.798737,291.309052,292.439514,...,3.162415e-08,7.947036e-12,7.859570e-12,3.934162e-11,3.740804e-08,5.115231e-08,4.202533,NaN,2003-01-01 12:00:00+00:00,"(15.0, -7.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228698465,0.000012,0.000713,0.002374,0.003769,0.004509,216.688309,248.984390,266.735138,277.806580,279.417542,...,4.352836e-09,1.946260e-10,8.113749e-12,0.000000e+00,4.004431e-09,5.632606e-09,12.724543,NaN,2020-12-31 09:00:00+00:00,"(47.0, 26.0)"
228698466,0.000018,0.000799,0.002521,0.003597,0.004745,216.353455,248.782974,266.835541,277.872803,280.625336,...,4.913757e-09,2.435661e-10,8.507474e-12,0.000000e+00,1.596592e-09,2.253301e-09,12.901739,NaN,2020-12-31 12:00:00+00:00,"(47.0, 26.0)"
228698467,0.000013,0.000789,0.002660,0.004196,0.004719,217.056976,248.452744,266.467041,277.363953,279.162201,...,3.724977e-09,1.456881e-10,4.253341e-12,0.000000e+00,2.267641e-09,3.152334e-09,13.212149,NaN,2020-12-31 15:00:00+00:00,"(47.0, 26.0)"
228698468,0.000013,0.000622,0.002176,0.004452,0.004585,217.388626,248.154755,265.953552,276.702606,277.759766,...,2.536197e-09,4.781012e-11,-7.915603e-16,0.000000e+00,2.938690e-09,4.051367e-09,13.522558,NaN,2020-12-31 18:00:00+00:00,"(47.0, 26.0)"


In [16]:
del sliced_tensor
del tensor
del dfs
del reshaped_tensor
df = (df.set_axis(channels46 +['stn_mesm','time','lat_lon_coord'],axis=1 )
      .drop('pm2p5',axis=1)
     )


In [17]:

def get_memory_usage_gb(variable):
    size_in_bytes = sys.getsizeof(variable)
    size_in_gb = size_in_bytes / (1024 ** 3)
    return size_in_gb


memory_usage_gb = get_memory_usage_gb(df)
print(f"Memory usage of variable: {memory_usage_gb:.2f} GB")

Memory usage of variable: 94.72 GB


In [19]:
# df.to_parquet('df.parquet.gzip',
#               compression='gzip')

In [2]:
df = pd.read_parquet('df.parquet.gzip')

In [18]:
# df.to_csv('test_before_features.csv',index=False)

In [3]:
grouped = df.groupby('lat_lon_coord')

# Creating new columns with previous row values for each group
for column in df.drop(['stn_mesm','time','lat_lon_coord'],axis=1).columns:
    new_column_name = column + '_prev'
    df[new_column_name] = grouped[column].shift()
    
df['week'] = df['time'].dt.strftime('%W').astype('int')
df['hour'] = df['time'].dt.hour
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year
df['sin'] = np.sin(df['time'].dt.dayofyear * 2 * np.pi / 365)
df['cos'] = np.cos(df['time'].dt.dayofyear * 2 * np.pi / 365)

del grouped

In [4]:
def add_delta_columns(df):
    channels_used =  ["Q250","Q500","Q700","Q850","Q900",
            "T250","T500","T700","T850","T900",
            "U250","U500","U700","U850","U900",
            "V250","V500","V700","V850","V900",
            "Z250","Z500","Z700","Z850","Z900",
            "PV250","PV500","PV700","PV850","PV900",
            "OMEGA250","OMEGA500","OMEGA700","OMEGA850","OMEGA900",
            "SLP","u10","v10","duaod550",'aermr06_50','aermr06_40','aermr06_30', 'aermr06_20',
              "tcwv"]

    # Loop over the channels names
    for col in channels_used:
        
        # Calculate the delta between the column and its corresponding "_prev" column
        delta_col = col + '_delta'
        prev_col = col + '_prev'
        df[delta_col] = df[col] - df[prev_col]
    
    return df

df = add_delta_columns(df)
df = df.drop(['pm10_prev'],axis=1)
df['pm10'] = df['pm10'] * (10**9)
df['target'] = df['stn_mesm'] - df['pm10']

# After Extanding test, load model and predict

In [7]:
def convert_string_to_tuple_list(string_list):
    tuple_list = []
    for string in string_list:
        # Remove leading and trailing whitespace and parentheses
        string = string.strip('()')
        # Convert string to tuple using ast.literal_eval
        tuple_value = ast.literal_eval(string)
        tuple_list.append(tuple_value)
    return tuple_list

def get_borders(coordinates):
    if not coordinates:
        return None

    # Initialize with the first coordinate in the list
    min_lat, max_lat = coordinates[0][0], coordinates[0][0]
    min_lon, max_lon = coordinates[0][1], coordinates[0][1]

    # Find the minimum and maximum latitude and longitude
    for lat, lon in coordinates:
        min_lat = min(min_lat, lat)
        max_lat = max(max_lat, lat)
        min_lon = min(min_lon, lon)
        max_lon = max(max_lon, lon)

    # Return the four corner points as a list of tuples
    top_left = (max_lat, min_lon)
    top_right = (max_lat, max_lon)
    bottom_left = (min_lat, min_lon)
    bottom_right = (min_lat, max_lon)

    return [top_left, top_right, bottom_left, bottom_right]

In [9]:
get_borders(convert_string_to_tuple_list(df['lat_lon_coord'].unique().tolist()))

[(47.0, -7.0), (47.0, 26.0), (15.0, -7.0), (15.0, 26.0)]

In [2]:
channels_used =  ["Q250","Q500","Q700","Q850","Q900",
            "T250","T500","T700","T850","T900",
            "U250","U500","U700","U850","U900",
            "V250","V500","V700","V850","V900",
            "Z250","Z500","Z700","Z850","Z900",
            "PV250","PV500","PV700","PV850","PV900",
            "OMEGA250","OMEGA500","OMEGA700","OMEGA850","OMEGA900",
            "SLP","u10","v10","duaod550",'aermr06_50','aermr06_40','aermr06_30', 'aermr06_20',
              "tcwv"]


previous_channels = [name+"_prev" for name in channels_used]
delta_channels = [name+'_delta' for name in channels_used]
time_features = ['week','hour','month','year','sin','cos']
last_features = ['pm10','stn_mesm','target','lat_lon_coord']


features = time_features + channels_used + previous_channels + delta_channels +['NDVI']



In [ ]:
df = pd.read_parquet('df.parquet_full_test.gzip')

In [11]:
df['NDVI'] = ndvi[0].values

In [8]:

model = xgb.Booster()
model.load_model("xgboost_model_trained_on_turkey_prevTime_Delta_NDVI.json")

In [12]:
num_rows = len(df) // 4

# Split the DataFrame into four parts based on row indices
df1 = df.iloc[:num_rows]
# df2 = df.iloc[num_rows: 2 * num_rows]
# df3 = df.iloc[2 * num_rows: 3 * num_rows]
# df4 = df.iloc[3 * num_rows:]

In [20]:
df2 = df.iloc[num_rows: 2 * num_rows]

In [25]:
df3 = df.iloc[2 * num_rows: 3 * num_rows]

In [28]:
df4 = df.iloc[3 * num_rows:]

In [ ]:
# df1.to_parquet('df_part1.parquet.gzip', compression='gzip')
           

In [25]:
split_index = int(0.5 * len(df))  
# Split the DataFrame
df_part1 = df.iloc[:split_index]
# df_part2 = df.iloc[split_index:]


In [15]:
delta1 = model.predict(xg.DMatrix(df1[features]))

In [21]:
delta2 = model.predict(xg.DMatrix(df2[features]))

In [31]:
len(delta1) + len(delta2) + len(delta3) + len(delta4)

228698470

In [24]:
del df1
del df2

In [26]:
delta3 = model.predict(xg.DMatrix(df3[features]))

In [27]:
del df3

In [29]:
delta4 = model.predict(xg.DMatrix(df4[features]))

In [32]:
delta_list = np.concatenate((delta1, delta2, delta3, delta4))

In [35]:
df['delta'] = delta_list

/tmp/ipykernel_34494/1365671935.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['delta'] = delta_list


In [36]:
df.head()

,Q250,Q500,Q700,Q850,Q900,T250,T500,T700,T850,T900,...,v10_delta,duaod550_delta,aermr06_50_delta,aermr06_40_delta,aermr06_30_delta,aermr06_20_delta,tcwv_delta,target,NDVI,delta
0,0.000013,0.000474,0.001102,0.000835,0.001759,226.508026,263.001862,283.515350,291.670929,293.279907,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.047821
1,0.000012,0.000481,0.001061,0.000656,0.001770,226.306519,262.616394,283.216797,291.796021,292.988007,...,0.290777,0.003501,-1.766947e-09,-1.145879e-12,-2.592444e-13,1.661348e-12,-0.205822,NaN,0.0,21.225750
2,0.000012,0.000364,0.001008,0.000889,0.001820,226.277267,262.529816,282.838684,291.154480,292.559235,...,0.290777,0.003501,-1.766947e-09,-1.145879e-12,-2.592444e-13,1.661352e-12,-0.205822,NaN,0.0,32.248775
3,0.000010,0.000294,0.000910,0.000942,0.001694,226.691376,262.407745,282.387604,290.868073,292.393738,...,0.133692,-0.020686,6.022267e-09,-8.744080e-12,7.694323e-13,6.981776e-13,0.028268,NaN,0.0,37.717949
4,0.000009,0.000176,0.000806,0.000866,0.001724,226.552704,263.003693,282.798737,291.309052,292.439514,...,0.133692,-0.020686,6.022269e-09,-8.744081e-12,7.694323e-13,6.981742e-13,0.028268,NaN,0.0,36.743103


In [37]:
df[['time','lat_lon_coord','delta','pm10','stn_mesm']].to_parquet('extened_test_small_ndvi.parquet_before_ndvi.gzip',
              compression='gzip')

In [ ]:
df[['time','lat_lon_coord','delta','pm10','stn_mesm']].to_csv('extened_test_small_ndvi.csv',index=False)

In [16]:
df.shape

(228698470, 144)

In [38]:
df.to_parquet('df.parquet_full_test.gzip',
              compression='gzip')